#### Code to validate models

In [136]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [137]:
cd ../src

/home/theo/kaggle/siim_covid/src


## Imports

In [138]:
import os
import cv2
import json
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import *
from collections import Counter
from tqdm.notebook import tqdm

In [139]:
from params import *

In [140]:
from data.preparation import prepare_dataframe, handle_duplicates
from data.dataset import CovidDetDataset, CovidClsDataset
from data.transforms import get_transfos_det, get_transfos_cls

from model_zoo.models import get_model

from utils.plot import plot_sample

from utils.logger import Config

from utils.metrics import per_class_average_precision_score, study_level_map

## Data

### Load

In [222]:
EXP_FOLDERS = [
    LOG_PATH + "2021-07-30/4/",
    LOG_PATH + "2021-07-31/0/",
#     LOG_PATH + "2021-08-01/0/",
    LOG_PATH + "2021-08-01/1/",
]

EXP_FOLDER = EXP_FOLDERS[-1]

USE_TTA = True

In [223]:
config = Config(json.load(open(EXP_FOLDER + "config.json", 'r')))

In [224]:
if USE_TTA:
    pred_oof_img = np.mean([np.load(f + "pred_oof_img_flip.npy") for f in EXP_FOLDERS], 0)
    pred_oof_study = np.mean([np.load(f + "pred_oof_study_flip.npy") for f in EXP_FOLDERS], 0)
else:
    pred_oof_img = np.mean([np.load(f + "pred_oof_img.npy") for f in EXP_FOLDERS], 0)
    pred_oof_study = np.mean([np.load(f + "pred_oof_study.npy") for f in EXP_FOLDERS], 0)

In [225]:
df = pd.read_csv(EXP_FOLDER + "data.csv")

pred_cols = [c + "_pred" for c in CLASSES]
df[pred_cols] = pred_oof_study
df['pred_img'] = pred_oof_img

In [226]:
ap = per_class_average_precision_score(pred_oof_img, df['img_target'].values)
print(f'Image mAP : {ap :.3f}')

auc = roc_auc_score(df['img_target'], pred_oof_img)
print(f'Image AUC : {auc :.3f}')
acc = accuracy_score(df['img_target'], pred_oof_img > 0.5)
print(f'Image Acc : {acc :.3f}')

Image mAP : 0.962
Image AUC : 0.912
Image Acc : 0.846


In [227]:
study_map = study_level_map(df[pred_cols].values, df[CLASSES].values, df['study_id'].values)
print(f'Study mAP : {study_map :.4f}')

Study mAP : 0.3837


### Post-process

#### Img merging

In [40]:
groups = df[['study_id', 'pred_img']].groupby('study_id').mean().rename(
    columns={'pred_img': 'pred_img_merged'}
).reset_index()
df_ = df.merge(groups, on="study_id", how="left")

df_.loc[df_['negative_pred'] > 0.75, 'pred_img'] = 0
df_.loc[df_['typical_pred'] > 0.75, 'pred_img'] = 1
df_.loc[df_['indeterminate_pred'] > 0.75, 'pred_img'] = 1

In [41]:
auc = roc_auc_score(df_['img_target'], df_['pred_img'])
print(f'Image AUC : {auc :.3f}')
acc = accuracy_score(df_['img_target'], df_['pred_img'] > 0.5)
print(f'Image Acc : {acc :.4f}')

Image AUC : 0.905
Image Acc : 0.8465


#### Study using img

In [42]:
df_ = df.copy()

p = 0.5
df_['negative_pred'] *= (1 - df_['pred_img']) ** p
df_['typical_pred'] *= (df_['pred_img']) ** p

# df_['indeterminate_pred'] *= (df_['pred_img']) ** p
# df_['atypical_pred'] *= (df_['pred_img']) ** p

study_map = study_level_map(df_[pred_cols].values, df_[CLASSES].values, df['study_id'].values)
print(f'Study mAP : {study_map :.5f}')

Study mAP : 0.38327


In [43]:
df_study = df[
    ['study_id'] + pred_cols + CLASSES + ['img_target', 'pred_img']
].groupby('study_id').agg(np.mean).copy()

# df_study['negative_pred'] *= 1 - df_study['pred_img'] 
# df_study['typical_pred'] *= df_study['pred_img'] 
# df_study['indeterminate_pred'] *= df_study['pred_img'] 
# df_study['atypical_pred'] *= df_study['pred_img'] 

df_study.loc[df_study['pred_img'] > 0.75, 'negative_pred'] *= 0.5
df_study.loc[df_study['pred_img'] < 0.2, 'negative_pred'] *= 2

df_study.loc[df_study['pred_img'] > 0.75, 'typical_pred'] *= 1.2
# df_study.loc[df_study['pred_img'] > 0.9, 'indeterminate_pred'] *= 1.1
# df_study.loc[df_study['pred_img'] > 0.9, 'atypical_pred'] *= 1.1

df_study.loc[df_study['pred_img'] < 0.25, 'typical_pred'] *= 0.8
# df_study.loc[df_study['pred_img'] < 0.2, 'indeterminate_pred'] *= 0.9
# df_study.loc[df_study['pred_img'] < 0.2, 'atypical_pred'] *= 0.9

accs = per_class_average_precision_score(
    df_study[pred_cols].values,
    df_study[CLASSES].values, 
    num_classes=NUM_CLASSES, 
    average=False
)
np.round(accs, 4), np.round(np.mean(accs) * 2/3, 4)

(array([0.802 , 0.8478, 0.3069, 0.3421]), 0.3831)

## Results

In [44]:
def proc(x):
    x = x.split('0 0 1 1')[:4]
    x = [float(y.strip().split(' ')[1]) for y in x]
    return x

In [239]:
sub = pd.read_csv("../output/sub_0931.csv")

In [240]:
sub

,id,PredictionString,negative,typical,indeterminate,atypical
0,00188a671292_study,negative 0.8009114503860474 0 0 1 1 typical 0....,0.800911,0.027383,0.054812,0.016894
1,004bd59708be_study,negative 0.0055577514367178 0 0 1 1 typical 0....,0.005558,0.808095,0.073134,0.013214
2,00508faccd39_study,negative 0.5854789519309997 0 0 1 1 typical 0....,0.585479,0.096047,0.140624,0.077850
3,006486aa80b2_study,negative 0.2664635345339775 0 0 1 1 typical 0....,0.266464,0.246429,0.310248,0.076859
4,00655178fdfc_study,negative 0.5432932925224304 0 0 1 1 typical 0....,0.543293,0.150926,0.170696,0.035084
...,...,...,...,...,...,...
1209,ff1ba0e9aaf0_study,negative 0.2834496781229972 0 0 1 1 typical 0....,0.283450,0.353800,0.217058,0.045693
1210,ff2cc4de58c5_study,negative 0.0885573822632432 0 0 1 1 typical 0....,0.088557,0.138054,0.389064,0.284325
1211,ff2f0a744930_study,negative 0.0028401464549824 0 0 1 1 typical 0....,0.002840,0.839094,0.047694,0.010372
1212,ff88940dce8b_study,negative 0.5658159017562866 0 0 1 1 typical 0....,0.565816,0.157659,0.141102,0.035423


In [251]:
sub = pd.read_csv("../output/sub_0931.csv")
sub_study = sub[sub['id'].apply(lambda x: "study" in x)].copy()

pred_test = np.array(sub_study['PredictionString'].apply(proc).values.tolist())

for i, c in enumerate(CLASSES):
    sub_study[c] = pred_test[:, i]
    
sub_study.to_csv("../output/sub_0931_study.csv", index=False)

In [259]:
sub = pd.read_csv("../output/sub_0931.csv")

sub_img = sub[sub['id'].apply(lambda x: "study" not in x)].copy()
sub_img['none'] = sub_img['PredictionString'].apply(lambda x: float(x.split('none')[-1].strip().split(' ')[0]))

sub_img.to_csv("../output/sub_0931_img.csv", index=False)

In [45]:
def treat_oof(pred_oof_study):
    pred_oof_study['id'] = pred_oof_study['id'].apply(lambda x: x.split('_')[0])
    
    df_study = pd.read_csv(DATA_PATH + "train_study_level.csv")
    df_study['study_id'] = df_study['id'].apply(lambda x: x.split('_')[0])
    df_study = df_study.rename(columns={c: c.split(' ')[0].lower() for c in df_study.columns})

    df_study.drop('id', axis=1, inplace=True)

    df_study = df_study.merge(pred_oof_study, how="left", left_on="study_id", right_on="id").dropna()
    
    pred_oof_study = np.array(df_study['PredictionString'].apply(proc).values.tolist())
    
    df_study[pred_cols] = pred_oof_study
    
    df_g = df[['study_id'] + pred_cols].groupby('study_id').mean().reset_index()
    df_study = df_study.merge(df_g, how="left", left_on="id", right_on="study_id", suffixes=['', '_theo'])
    
    return df_study

In [46]:
df_old = treat_oof(pd.read_csv('../output/OOF_study_only_EBV2M_768.csv'))
df_v2m_2 = treat_oof(pd.read_csv('../output/oof_v2m.csv'))
df_b4 = treat_oof(pd.read_csv('../output/oof_b4.csv'))
df_b5 = treat_oof(pd.read_csv('../output/oof_b5.csv'))
df_ono = treat_oof(pd.read_csv('../output/oof_ono.csv')[['id', 'PredictionString']])

In [48]:
df_theo = df[['study_id'] + pred_cols + CLASSES].groupby('study_id').mean().reset_index()
df_theo = df_v2m_2[['id']].merge(df_theo, how="left", left_on="id", right_on="study_id")

df_ono = df_v2m_2[['id']].merge(df_ono, how="left", left_on="id", right_on="study_id")

In [116]:
p = 1
oofs = [
#     df_old[pred_cols].values ** p * 1,
    df_theo[pred_cols].values ** p * 1,
    df_v2m_2[pred_cols].values ** p * 1,
#     df_b4[pred_cols].values ** p * 0.1,
    df_b5[pred_cols].values ** p * 0.5,
#     df_ono[pred_cols].values ** p * 0.1,
]

oof = np.mean(oofs, 0)

In [117]:
oof.max(0)

array([0.81562984, 0.81986495, 0.45891351, 0.71444997])

In [118]:
CLASSES

['negative', 'typical', 'indeterminate', 'atypical']

In [122]:
from scipy.stats import pearsonr

In [135]:
for i in range(4):
    print(pearsonr(pred[:, 2], pred[:, i])[0])

-0.31101399733824325
-0.07768501668761119
1.0
0.4056669876265603


In [133]:
truth = df_theo[CLASSES].values
pred = oof
aucs = [roc_auc_score(truth[:, i].flatten(), pred[:, i].flatten()) for i in range(pred.shape[1])]
aucs

[0.9243023088403703,
 0.8713742063953793,
 0.6932290198865377,
 0.8066562284884381]

In [109]:
per_class_average_precision_score(oof, df_theo[CLASSES].values, num_classes=NUM_CLASSES) * 2 / 3

0.39189165578531077

In [110]:
per_class_average_precision_score(oof, df_theo[CLASSES].values, num_classes=NUM_CLASSES, average=False) #* 2 / 3

array([0.81115651, 0.85174298, 0.3206541 , 0.36779635])